## Make Positions

The positions in the GPX file occur approximately every eight hours. To get a smoother animation that allows the progression of the weather to be seen more frequent positions are required. This notebook takes the original fixes and then interpolates between the points to get hourly positions. Various interpolation techniques are possible; a simple linear interpolation based on the bearing and distance between fixes will be tried first.

In [1]:
from geographiclib.geodesic import Geodesic
from geopy import distance
import gpxpy
import pandas as pd

In [2]:
gpx_file = 'yblog-1589101179046.gpx'

In [3]:
with open(gpx_file) as fh:
    gpx = gpxpy.parse(fh)
data = gpx.tracks[0].segments[0].points

In [4]:
print(data[0])

[trkpt:55.94507999999999,-4.746780000000001@8.0@2013-09-04 16:19:01+00:00]


In [5]:
df = pd.DataFrame(columns=['lat', 'lon', 'time'])

# Skip the first entry as it was just a test one week before we departed
for point in data[1:]: 
    df = df.append({'lon': point.longitude, 'lat' : point.latitude, 'time' : point.time}, ignore_index=True)

In [6]:
print(df[:10])
print(df[-3:])


        lat      lon                       time
0  55.94507 -4.74655  2013-09-16 21:54:06+00:00
1  55.56232 -5.02120  2013-09-17 16:00:06+00:00
2  54.66902 -5.29117  2013-09-18 00:00:08+00:00
3  53.64186 -4.84571  2013-09-18 08:00:06+00:00
4  53.32146 -4.64268  2013-09-18 16:00:06+00:00
5  53.32146 -4.64260  2013-09-19 00:00:06+00:00
6  53.32148 -4.64269  2013-09-19 08:00:06+00:00
7  53.32148 -4.64269  2013-09-19 16:00:06+00:00
8  53.34245 -4.64908  2013-09-20 00:00:37+00:00
9  52.48326 -5.49176  2013-09-20 08:00:25+00:00
          lat      lon                       time
546  58.03371 -6.33969  2014-05-24 00:00:06+00:00
547  58.20922 -6.39012  2014-05-24 08:00:06+00:00
548  58.20922 -6.39012  2014-05-24 09:59:12+00:00


In [7]:
ms_to_knot = 1.944
geod = Geodesic.WGS84
hourly = pd.DataFrame(columns=['lat', 'lon', 'time', 'bearing', 'speed', 'src'])
# src is a character with meanings: f: fix
#                                   i: interpolated between fixes
#                                   m: manually inserted point (e.g. when 
#                                      tracker was off in port)
seconds_in_hour = 60**2
for orig_index in range(len(df.index) - 1):
    l = geod.InverseLine(
        df.loc[orig_index]['lat'],
        df.loc[orig_index]['lon'],
        df.loc[orig_index + 1]['lat'],
        df.loc[orig_index + 1]['lon'],
    )
    start_time = df.iloc[orig_index]['time']
    start_time_hour = start_time.replace(minute=0, second=0, microsecond=0)
    end_time = df.iloc[orig_index + 1]['time']
    end_time_hour = end_time.replace(minute=0, second=0, microsecond=0)
    time_delta_hours = int((end_time_hour - start_time_hour).total_seconds() / 
                           seconds_in_hour)
    time_delta = (end_time - start_time).total_seconds()
    speed = (l.s13 / time_delta) * ms_to_knot
    for i in range(time_delta_hours):
        g = l.Position(
            (l.s13 / time_delta_hours) * i, 
            Geodesic.STANDARD | Geodesic.LONG_UNROLL
        )
        hourly = hourly.append({
            'time': start_time_hour + pd.Timedelta(hours=i),
            'lat': g['lat2'],
            'lon': g['lon2'],
            'bearing': g['azi2'] if  g['azi2'] >= 0 else 360 +  g['azi2'],
            'speed': speed,
            'src': 'f' if i == 0 else 'i',
        }, ignore_index=True)

In [8]:
print(hourly[:24])
print(hourly[-24:])

          lat       lon                       time     bearing     speed src
0   55.945070 -4.746550  2013-09-16 21:00:00+00:00  202.145573  1.371521   f
1   55.924941 -4.761140  2013-09-16 22:00:00+00:00  202.133486  1.371521   i
2   55.904811 -4.775715  2013-09-16 23:00:00+00:00  202.121415  1.371521   i
3   55.884678 -4.790275  2013-09-17 00:00:00+00:00  202.109359  1.371521   i
4   55.864544 -4.804819  2013-09-17 01:00:00+00:00  202.097319  1.371521   i
5   55.844408 -4.819349  2013-09-17 02:00:00+00:00  202.085294  1.371521   i
6   55.824271 -4.833864  2013-09-17 03:00:00+00:00  202.073285  1.371521   i
7   55.804131 -4.848363  2013-09-17 04:00:00+00:00  202.061290  1.371521   i
8   55.783990 -4.862848  2013-09-17 05:00:00+00:00  202.049311  1.371521   i
9   55.763847 -4.877318  2013-09-17 06:00:00+00:00  202.037347  1.371521   i
10  55.743702 -4.891773  2013-09-17 07:00:00+00:00  202.025398  1.371521   i
11  55.723556 -4.906213  2013-09-17 08:00:00+00:00  202.013464  1.371521   i

In [9]:
output_name = 'hourly_positions.json'
with open(output_name, 'w') as fh:
    hourly.to_json(fh)